In [29]:
import requests
import socket
import pandas as pd

In [26]:
# обёртка над запросами в aviasales
def request_aviasales(action, params):
    response = requests.get(
        f'http://min-prices.aviasales.ru/{action}', 
        params=params
    )
    
    return response.json()

In [27]:
# обёртка над запросами в IATA-сервис
# можно сделать несколько обърток для каждого метода, но, т.к. мы будем запускать их по 1 разу, то в этом нет смысла
def request_iata(action, params):
    response = requests.get(
        f'http://www.travelpayouts.com/{action}', 
        params=params
    )
    
    return response.json()

In [37]:
ip = socket.gethostbyname(socket.gethostname()) # получаем наш IP

# пытаемся по IP узнать наш город
if ip != '127.0.0.1':
    current_city = request_iata('whereami', {
        'locale': 'ru', 
        'ip': ip
    })

# формируем предложение пользователю
to_request = 'Введите пункт оптравления:'
if current_city:
    to_request = f'Введите пункт оптравления (текущий {current_city["name"]}): '
    
origin = input(to_request) # запрашиваем пункт отправления
destination = input('Введите пункт назначения: ') # запрашиваем пункт назначения
one_way = input('В один конец (да/нет)? ') # запрашиваем необходимость поиска обратного билета

# если не указан пункт отправления, но есть данные по IP - используем их
if not origin and current_city:
    origin = current_city['name']

# получаем IATA коды для пунктов отправления и назначения
transport_data = request_iata(
    'widgets_suggest_params', 
    {
        'q': f'Из {origin} в {destination}'
    }
)

# делаем запрос за билетами
data = request_aviasales(
    'calendar_preload', {
        'origin': transport_data['origin']['iata'],
        'destination': transport_data['destination']['iata'],
        'one_way': one_way == 'да'
    })

{'locale': 'ru', 'ip': '192.168.0.85'}
Введите пункт оптравления (текущий Москва): Самара
Введите пункт назначения: Санкт-Петербург
В один конец (да/нет)? 
{'q': 'Из Самара в Санкт-Петербург'}


In [39]:
# выводим данные
# тут я схитрил и положился на DataFrame и Jupiter :)
df = pd.DataFrame(data['best_prices'])
df.rename(columns={
    'origin': 'Пункт отправления', 
    'destination': 'Пункт назначения', 
    'depart_date': 'Дата вылета', 
    'return_date': 'Дата возвращения', 
    'value': 'Цена',
    'number_of_changes': 'Кол-во пересадок'
}, inplace=True)

df[['Пункт отправления', 'Пункт назначения', 'Дата вылета', 'Дата возвращения', 'Кол-во пересадок', 'Цена']]

    Пункт отправления Пункт назначения Дата вылета Дата возвращения  \
0                 KUF              LED  2020-01-16       2020-01-31   
1                 KUF              LED  2019-12-23       2019-12-27   
2                 KUF              LED  2019-10-09       2019-10-10   
3                 KUF              LED  2019-11-26       2019-11-27   
4                 KUF              LED  2020-06-18       2020-06-20   
5                 KUF              LED  2020-01-10       2020-01-15   
6                 KUF              LED  2019-10-02       2019-10-16   
7                 KUF              LED  2020-01-04       2020-01-10   
8                 KUF              LED  2019-11-16       2019-11-18   
9                 KUF              LED  2019-12-03       2019-12-10   
10                KUF              LED  2019-09-11       2019-09-25   
11                KUF              LED  2019-10-20       2019-10-24   
12                KUF              LED  2019-10-15       2019-10-18   
13    